## This notebook reads in local xml files in bulk and assembles a dataset which it cleans and reduces and exports as a csv

In [2]:
# imports

import xml.etree.ElementTree as et 
import pandas as pd
pd.set_option('max_columns', 100)

import os
import time

import warnings
warnings.simplefilter(action = 'ignore', category = Warning)

In [51]:
# an empty list where we'll put just the dicts we want
selected = []

# keep a count of skipped empty files (0KB) and of ignored DSSTORE or jupyter files
skipped_ctr = 0
ignored_ctr = 0

# create a list of all the local files, the path to them and a file counter to increase
list_of_files = os.listdir('/Users/aidanair/Documents/DATA/V_ENV/virtual_env_tri/SPANISH_ARCHIVE_FILES')
file_ctr = 0
path = '/Users/aidanair/Documents/DATA/V_ENV/virtual_env_tri/SPANISH_ARCHIVE_FILES/'

# read through all the files
for x in range (len(list_of_files)):

    # define the file location and name
    local_file = path + list_of_files[file_ctr]
    
    # find the file size
    filesize = os.path.getsize(local_file)
    
    # if file is empty, skip it
    if filesize == 0:
        file_ctr += 1
        skipped_ctr += 1
        continue
        
    # if it's a DSSTORE or NOTEBOOK CHECKPOINT, skip it
    other_files = ['/Users/aidanair/Documents/DATA/V_ENV/virtual_env_tri/SPANISH_ARCHIVE_FILES/.DS_Store', '/Users/aidanair/Documents/DATA/V_ENV/virtual_env_tri/SPANISH_ARCHIVE_FILES/.ipynb_checkpoints']
    if local_file in other_files:
        file_ctr += 1
        ignored_ctr += 1
        continue 
        
    # otherwise:    
    else:
        
        # create a tree and root to parse the xml
        xtree = et.parse(local_file)
        xroot = xtree.getroot()
    
        # create empty dict
        empty = {}

        # define the list of k tags / keys we want in our shorter df
        key_list = ['recordid', 'otherrecordid', 'titleproper', 'agencycode', 'agencyname', 'fromdate', 'todate', 'url']

        # find all elements then collect the tags as keys, and the text as values
        for c in xroot.findall(".//"):
            k = c.tag.split('}')[1]
            v = c.text

            # if the key is in the selected list, add it to our empty dictionary
            if k in key_list:   
                empty[k] = v

        # define complete url (for the html page) to which this xml file corresponds and add to dict
        ending = local_file.split('_')[-1].split('.')[0]
        url = f'http://pares.mcu.es/ParesBusquedas20/catalogo/description/{ending}'
        empty['url'] = url

        # add our dictionary to our overall list of dicts
        selected.append(empty)
        
        # print a counter marker every 5000
        if file_ctr % 5000 == 0:
            print("file counter currently at:", file_ctr)
            
        # increase counter
        file_ctr += 1

# when finished, show the number of dictionaries now in the list (and how many 0KB files or unwanted files were skipped):

print(len(selected))
print(skipped_ctr)
print(ignored_ctr)

file counter currently at: 0
file counter currently at: 5000
file counter currently at: 10000
file counter currently at: 15000
file counter currently at: 20000
file counter currently at: 25000
file counter currently at: 30000
file counter currently at: 35000
file counter currently at: 40000
41442
788
2


In [52]:
# make the df

df = pd.DataFrame(selected)
print(df.shape)
df[:4]

(41442, 8)


,recordid,otherrecordid,titleproper,agencycode,agencyname,fromdate,todate,url
0,"ES.37274.CDMH/8.4.8.12//DNSD-SECRETARIA,FICHER...",ES-CDMH-37274-10104386,Ficha de Maria Amago,ES-37008-CDMH,CDMH,1937,1977,http://pares.mcu.es/ParesBusquedas20/catalogo/...
1,"ES.37274.CDMH/8.4.8.12//DNSD-SECRETARIA,FICHER...",ES-CDMH-37274-10094959,Ficha de Antonio Alonso Prieto,ES-37008-CDMH,CDMH,1937,1977,http://pares.mcu.es/ParesBusquedas20/catalogo/...
2,"ES.37274.CDMH/8.4.8.12//DNSD-SECRETARIA,FICHER...",ES-CDMH-37274-10079437,Ficha de Manuel Adrio Seijas,ES-37008-CDMH,CDMH,1937,1977,http://pares.mcu.es/ParesBusquedas20/catalogo/...
3,"ES.37274.CDMH/8.4.8.12//DNSD-SECRETARIA,FICHER...",ES-CDMH-37274-10114195,Ficha de Marcelino Antolin,ES-37008-CDMH,CDMH,1937,1977,http://pares.mcu.es/ParesBusquedas20/catalogo/...


## clean the df

In [53]:
# remove 'Ficha de ' from in front of each name

def remove_ficha(titleproper):
    
    splitup = titleproper.split('Ficha de ')
    return splitup[1]

df['name'] = df.titleproper.apply(remove_ficha)

In [54]:
# move the id col to the end in order to split it easily

df = df[['name', 'otherrecordid', 'titleproper', 'agencycode', 'agencyname', 'fromdate', 'todate', 'url', 'recordid']]


In [55]:
# check

df[:5]

,name,otherrecordid,titleproper,agencycode,agencyname,fromdate,todate,url,recordid
0,Maria Amago,ES-CDMH-37274-10104386,Ficha de Maria Amago,ES-37008-CDMH,CDMH,1937,1977,http://pares.mcu.es/ParesBusquedas20/catalogo/...,"ES.37274.CDMH/8.4.8.12//DNSD-SECRETARIA,FICHER..."
1,Antonio Alonso Prieto,ES-CDMH-37274-10094959,Ficha de Antonio Alonso Prieto,ES-37008-CDMH,CDMH,1937,1977,http://pares.mcu.es/ParesBusquedas20/catalogo/...,"ES.37274.CDMH/8.4.8.12//DNSD-SECRETARIA,FICHER..."
2,Manuel Adrio Seijas,ES-CDMH-37274-10079437,Ficha de Manuel Adrio Seijas,ES-37008-CDMH,CDMH,1937,1977,http://pares.mcu.es/ParesBusquedas20/catalogo/...,"ES.37274.CDMH/8.4.8.12//DNSD-SECRETARIA,FICHER..."
3,Marcelino Antolin,ES-CDMH-37274-10114195,Ficha de Marcelino Antolin,ES-37008-CDMH,CDMH,1937,1977,http://pares.mcu.es/ParesBusquedas20/catalogo/...,"ES.37274.CDMH/8.4.8.12//DNSD-SECRETARIA,FICHER..."
4,German Abrahan,ES-CDMH-37274-10076704,Ficha de German Abrahan,ES-37008-CDMH,CDMH,1937,1977,http://pares.mcu.es/ParesBusquedas20/catalogo/...,"ES.37274.CDMH/8.4.8.12//DNSD-SECRETARIA,FICHER..."


In [56]:
# split the first part of record id (keeping original) using /

df[['recordid_a', 'recordid_b', 'recordid_x', 'recordid_c']] = df.recordid.str.split("/", expand=True)

In [57]:
# split remainder of record id using ,

df[['recordid_d', 'recordid_e', 'recordid_f', 'recordid_g']] = df.recordid.str.split(",", expand=True)

In [58]:
# check

df[:3]

,name,otherrecordid,titleproper,agencycode,agencyname,fromdate,todate,url,recordid,recordid_a,recordid_b,recordid_x,recordid_c,recordid_d,recordid_e,recordid_f,recordid_g
0,Maria Amago,ES-CDMH-37274-10104386,Ficha de Maria Amago,ES-37008-CDMH,CDMH,1937,1977,http://pares.mcu.es/ParesBusquedas20/catalogo/...,"ES.37274.CDMH/8.4.8.12//DNSD-SECRETARIA,FICHER...",ES.37274.CDMH,8.4.8.12,,"DNSD-SECRETARIA,FICHERO,3,A0101002",ES.37274.CDMH/8.4.8.12//DNSD-SECRETARIA,FICHERO,3,A0101002
1,Antonio Alonso Prieto,ES-CDMH-37274-10094959,Ficha de Antonio Alonso Prieto,ES-37008-CDMH,CDMH,1937,1977,http://pares.mcu.es/ParesBusquedas20/catalogo/...,"ES.37274.CDMH/8.4.8.12//DNSD-SECRETARIA,FICHER...",ES.37274.CDMH,8.4.8.12,,"DNSD-SECRETARIA,FICHERO,2,A0062280",ES.37274.CDMH/8.4.8.12//DNSD-SECRETARIA,FICHERO,2,A0062280
2,Manuel Adrio Seijas,ES-CDMH-37274-10079437,Ficha de Manuel Adrio Seijas,ES-37008-CDMH,CDMH,1937,1977,http://pares.mcu.es/ParesBusquedas20/catalogo/...,"ES.37274.CDMH/8.4.8.12//DNSD-SECRETARIA,FICHER...",ES.37274.CDMH,8.4.8.12,,"DNSD-SECRETARIA,FICHERO,1,A0011650",ES.37274.CDMH/8.4.8.12//DNSD-SECRETARIA,FICHERO,1,A0011650


In [59]:
# reorder the cols, also removing any duplicate or unnecessary cols

df = df[['name', 'recordid_a', 'recordid_b', 'recordid_e', 'recordid_f', 'recordid_g', 'fromdate', 'todate', 'agencycode', 'agencyname', 'recordid', 'otherrecordid', 'url']]


In [60]:
# rename the cols

df.columns = ['name', 'recordid_a', 'recordid_b', 'recordid_c', 'recordid_d', 'recordid_e', 'fromdate', 'todate', 'agencycode', 'agencyname', 'recordid', 'otherrecordid', 'url']


In [61]:
# finished

df[:3]

,name,recordid_a,recordid_b,recordid_c,recordid_d,recordid_e,fromdate,todate,agencycode,agencyname,recordid,otherrecordid,url
0,Maria Amago,ES.37274.CDMH,8.4.8.12,FICHERO,3,A0101002,1937,1977,ES-37008-CDMH,CDMH,"ES.37274.CDMH/8.4.8.12//DNSD-SECRETARIA,FICHER...",ES-CDMH-37274-10104386,http://pares.mcu.es/ParesBusquedas20/catalogo/...
1,Antonio Alonso Prieto,ES.37274.CDMH,8.4.8.12,FICHERO,2,A0062280,1937,1977,ES-37008-CDMH,CDMH,"ES.37274.CDMH/8.4.8.12//DNSD-SECRETARIA,FICHER...",ES-CDMH-37274-10094959,http://pares.mcu.es/ParesBusquedas20/catalogo/...
2,Manuel Adrio Seijas,ES.37274.CDMH,8.4.8.12,FICHERO,1,A0011650,1937,1977,ES-37008-CDMH,CDMH,"ES.37274.CDMH/8.4.8.12//DNSD-SECRETARIA,FICHER...",ES-CDMH-37274-10079437,http://pares.mcu.es/ParesBusquedas20/catalogo/...


In [62]:
## check for duplicate rows in the urls column. No url should appear more than once

df.url.value_counts()

http://pares.mcu.es/ParesBusquedas20/catalogo/description/10081430    1
http://pares.mcu.es/ParesBusquedas20/catalogo/description/10109013    1
http://pares.mcu.es/ParesBusquedas20/catalogo/description/10080924    1
http://pares.mcu.es/ParesBusquedas20/catalogo/description/10111157    1
http://pares.mcu.es/ParesBusquedas20/catalogo/description/10103184    1
                                                                     ..
http://pares.mcu.es/ParesBusquedas20/catalogo/description/10105182    1
http://pares.mcu.es/ParesBusquedas20/catalogo/description/10105576    1
http://pares.mcu.es/ParesBusquedas20/catalogo/description/10082304    1
http://pares.mcu.es/ParesBusquedas20/catalogo/description/10114066    1
http://pares.mcu.es/ParesBusquedas20/catalogo/description/10075647    1
Name: url, Length: 41442, dtype: int64

In [66]:
# export as a csv, with no index - with first and last file number and date and total length of df

df.to_csv('/Users/aidanair/Desktop/pares_archive_10_074_724_10_117_072_03042021_LEN41442.csv', index = False)